In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
import glob

import numpy as np
import pylab as plt

from scipy.stats import norm
from scipy.optimize import curve_fit
from astropy.stats import sigma_clip

from astropy.io import fits
from  natsort import natsorted

import sys
sys.path.append('/home/scratch/psalas/LASSI/lassi-analysis_v2')
from analyzeActiveSurface import processActiveSurfaceFITSPair

In [3]:
scan0 = 16
scanf = 125
n_zern = 37
path = "/home/gbtdata/TLASSI_200315"

In [4]:
# Use scan 16 up to 125.
# Scans 126 onwards are two reflector data.
hdu = fits.open("{0}/ScanLog.fits".format(path))
scanArr = hdu[1].data

In [5]:
idx0 = np.where(scanArr['SCAN'] == scan0)[0]
idxf = np.where(scanArr['SCAN'] == scanf)[0]

In [6]:
zern_files = natsorted(glob.glob("{0}/LASSI/*.zernike.fits".format(path)))[3:]

In [7]:
z_idx = np.zeros((len(zern_files)), dtype=np.int)
scans = np.zeros((len(zern_files)), dtype=np.int)

z_as = np.zeros((len(zern_files), n_zern-1), dtype=np.float)
z_lassi = np.zeros((len(zern_files), n_zern), dtype=np.float)
z_as_obs = np.zeros((len(zern_files), n_zern), dtype=np.float)

for i, zf in enumerate(zern_files):
    
    hdu = fits.open(zf)
    head = hdu[0].header
    z_lassi[i] = hdu[2].data['value']
    scans[i] = head['MC_SCAN']
    
    idx = np.where(scanArr['SCAN'] == scans[i])[0][0]
    idx_as = idx - 3

    # Load the AS fits file for the signal scan.
    as_file = file = '/'.join(scanArr[idx_as]['FILEPATH'].split('/')[-2:])
    hdu = fits.open("{0}/{1}".format(path, as_file))
    z_as[i] = hdu[1].data['value']
    
    # The AS Zernike contents start at Z1, not Z0.
    # The LASSI Zernike start at Z0.
    z_idx[i] = np.where(z_as[i] != 0)[0]
    
    as_file_sig = '/'.join(scanArr[idx_as]['FILEPATH'].split('/')[-2:])
    as_file_ref = '/'.join(scanArr[idx_as - 6]['FILEPATH'].split('/')[-2:])
    print(as_file_sig, as_file_ref)
    aas, aar, fitlist = processActiveSurfaceFITSPair("{0}/{1}".format(path, as_file_ref), 
                                                     "{0}/{1}".format(path, as_file_sig), 
                                                      column='ABSOLUTE', filterDisabled=True, 
                                                      verbose=False, plot=False)
    z_as_obs[i] = fitlist

ActiveSurfaceMgr/2020_03_16_02:28:17.fits ActiveSurfaceMgr/2020_03_16_02:20:53.fits
ActiveSurfaceMgr/2020_03_16_03:02:38.fits ActiveSurfaceMgr/2020_03_16_02:54:45.fits
ActiveSurfaceMgr/2020_03_16_03:27:11.fits ActiveSurfaceMgr/2020_03_16_03:19:37.fits
ActiveSurfaceMgr/2020_03_16_03:51:48.fits ActiveSurfaceMgr/2020_03_16_03:44:12.fits
ActiveSurfaceMgr/2020_03_16_04:16:26.fits ActiveSurfaceMgr/2020_03_16_04:08:53.fits
ActiveSurfaceMgr/2020_03_16_04:40:56.fits ActiveSurfaceMgr/2020_03_16_04:33:20.fits
ActiveSurfaceMgr/2020_03_16_05:05:30.fits ActiveSurfaceMgr/2020_03_16_04:57:50.fits
ActiveSurfaceMgr/2020_03_16_05:29:51.fits ActiveSurfaceMgr/2020_03_16_05:22:11.fits
ActiveSurfaceMgr/2020_03_16_05:54:22.fits ActiveSurfaceMgr/2020_03_16_05:46:56.fits
ActiveSurfaceMgr/2020_03_16_06:18:40.fits ActiveSurfaceMgr/2020_03_16_06:11:07.fits
ActiveSurfaceMgr/2020_03_16_06:43:03.fits ActiveSurfaceMgr/2020_03_16_06:35:29.fits
ActiveSurfaceMgr/2020_03_16_07:07:35.fits ActiveSurfaceMgr/2020_03_16_07:00:

In [8]:
z_in = np.zeros(len(zern_files), dtype=np.float)
z_in_as = np.zeros(len(zern_files), dtype=np.float)
z_obs = np.zeros(len(zern_files), dtype=np.float)

for i in range(len(zern_files)):
    z_in[i] = z_as[i][z_idx[i]]
    z_in_as[i] = z_as_obs[i][z_idx[i]+1]*1e6
    z_obs[i] = z_lassi[i][z_idx[i]+1]

In [10]:
# Compute statistics per coefficient, per polynomial
zobs_x = np.unique(z_in)
uiz = np.unique(z_idx)

zobs_mu_vz = np.zeros((len(zobs_x),len(uiz)), dtype=np.float)
zobs_sigma_vz = np.zeros((len(zobs_x),len(uiz)), dtype=np.float)
zobs_stds_vz = np.zeros((len(zobs_x),len(uiz)), dtype=np.float)
zobs_means_vz = np.zeros((len(zobs_x),len(uiz)), dtype=np.float)
zobs_pdiff_vz = np.zeros((len(zobs_x),len(uiz)), dtype=np.float)
zobs_diff = np.zeros((len(zobs_x),len(uiz)), dtype=np.float)

for i,izv_ in enumerate(zobs_x):
    for j,iz_ in enumerate(uiz):
        
        mask = (z_in == izv_) & (z_idx == iz_)
        
        izvas = izv_
        izvas = abs(np.nanmean(z_in_as[mask]))
        
        zobs_stds_vz[i,j] = np.nanstd(z_obs[mask])
        zobs_means_vz[i,j] = np.nanmean(z_obs[mask])
        zobs_pdiff_vz[i,j] = (zobs_means_vz[i,j] - izvas)/izvas
        zobs_diff[i,j] = (zobs_means_vz[i,j] - izvas)
        print(izv_, iz_+1, mask.sum(), 
              "izv={0:.1f}".format(izvas), 
              'mean={0:.1f}'.format(zobs_means_vz[i,j]), 
              "  {0:.1f} %  ".format(zobs_pdiff_vz[i,j]*100.),
              'diff={0:.1f}'.format(zobs_diff[i,j]),)

70.0 4 2 izv=74.1 mean=61.4   -17.2 %   diff=-12.7
70.0 7 2 izv=67.7 mean=88.2   30.2 %   diff=20.5
70.0 13 1 izv=54.7 mean=18.8   -65.6 %   diff=-35.9
150.0 4 2 izv=177.3 mean=325.9   83.8 %   diff=148.6
150.0 7 2 izv=158.0 mean=240.1   52.0 %   diff=82.1
150.0 13 1 izv=165.1 mean=171.2   3.7 %   diff=6.1
500.0 4 3 izv=510.2 mean=472.9   -7.3 %   diff=-37.3
500.0 7 2 izv=527.7 mean=498.6   -5.5 %   diff=-29.1
500.0 13 2 izv=520.7 mean=455.0   -12.6 %   diff=-65.6


In [30]:
mask = (z_idx+1 == 13) & (z_in == 500.)
print(z_idx[mask]+1)
print(z_in_as[mask], abs(z_in_as[mask]).mean())
print(z_obs[mask], z_obs[mask].mean())
print((z_obs[mask]-abs(z_in_as[mask]))/abs(z_in_as[mask])*100.) 
print(scans[mask])
print(np.where(mask==True))
for i in range(mask.sum()):
    print(scanArr[np.where(scans[mask][i] == scanArr['SCAN'])[0][0]])

[13 13]
[520.94659528 520.36996366] 520.6582794672106
[472.12939728 437.89174364] 455.01057045663003
[ -9.3708642  -15.84991944]
[ 63 117]
(array([ 7, 16]),)
('2020-03-16T05:30:05', 63, './TLASSI_200315/LASSI/2020_03_16_05:30:05.fits')
('2020-03-16T09:09:14', 117, './TLASSI_200315/LASSI/2020_03_16_09:09:14.fits')


In [40]:
# Z13=500
print(np.mean(((528.-520.94659528)/520.94659528*100., 
               (466.-520.36996366)/520.36996366*100.)))
print(np.mean(((501.-520.94659528)/520.94659528*100., 
               (480.-520.36996366)/520.36996366*100.)))
# Z4=150
print(np.mean(((298.-162.51285227)/162.51285227*100., 
               (325.-192.08927394)/192.08927394*100.)))
# Z7=150
print(np.mean(((248.-142.22445901)/142.22445901*100., 
               (311.-173.759286)/173.759286*100.)))

-4.547184657279264
-5.793424334979922
76.28113839792523
76.67774882112755
